In [472]:
import pandas as pd

xlsx = pd.ExcelFile('../Downloads/Moving annual rent by suburb - March quarter 2023.xlsx')

dfs = []

for sheet_name in xlsx.sheet_names:

    df = pd.read_excel(xlsx, sheet_name=sheet_name, header=1)
    df = df.iloc[:, 1:]
    df = df.drop(0, axis=0)
    
    df.columns = ['Suburb'] + df.columns[1:].tolist()
    
    date_columns = df.columns[1::2]  
    unnamed_columns = df.columns[2::2]  

    rename_dict = {}
    for date_col, unnamed_col in zip(date_columns, unnamed_columns):
        if 'Unnamed' in unnamed_col:
            new_name = f"{date_col}_median"  
            rename_dict[unnamed_col] = new_name
    
    df = df.rename(columns=rename_dict)
    
    df['property type'] = sheet_name
    
    date_rename = {}
    for date_col in date_columns:
        count_name = f"{date_col}_count"
        date_rename[date_col] = count_name
    
    df = df.rename(columns=date_rename)

    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

In [473]:
merged_df

,Suburb,Mar 2000_count,Mar 2000_median,Jun 2000_count,Jun 2000_median,Sep 2000_count,Sep 2000_median,Dec 2000_count,Dec 2000_median,Mar 2001_count,...,Mar 2022_median,Jun 2022_count,Jun 2022_median,Sep 2022_count,Sep 2022_median,Dec 2022_count,Dec 2022_median,Mar 2023_count,Mar 2023_median,property type
0,Albert Park-Middle Park-West St Kilda,352,165,347,165,378,170,369,175,395,...,315,262,325,277,340,260,350,262,360,1 bedroom flat
1,Armadale,210,150,212,150,213,155,213,160,206,...,310,233,320,237,340,238,350,204,360,1 bedroom flat
2,Carlton North,87,150,78,155,74,150,65,150,73,...,300,68,320,72,320,71,330,64,373,1 bedroom flat
3,Carlton-Parkville,303,165,300,170,315,175,351,180,360,...,259,3544,269,4296,300,4203,309,4488,352,1 bedroom flat
4,CBD-St Kilda Rd,755,250,861,250,934,250,952,250,1145,...,320,8619,340,9913,365,9638,390,9781,430,1 bedroom flat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,Wanagaratta,705,125,671,125,631,130,623,130,602,...,370,530,380,547,380,541,380,518,380,All properties
1109,Warragul,385,130,367,135,382,135,366,135,365,...,400,512,400,515,420,523,430,497,440,All properties
1110,Warrnambool,1266,130,1229,135,1204,135,1135,135,1069,...,400,844,400,870,420,859,420,867,420,All properties
1111,Wodonga,1446,145,1439,145,1468,150,1449,150,1405,...,380,1267,390,1251,400,1191,410,1173,410,All properties


In [474]:
df = pd.DataFrame(merged_df)

# Function to reshape DataFrame
def reshape_data(df):
    reshaped_data = []

    # Iterate through each row in the DataFrame
    for _, row in df.iterrows():
        suburb = row['Suburb']
        property_type = row['property type']
        
        # For each count and median column, add a new row to reshaped_data
        for col in df.columns:
            if '_count' in col:
                date = col.replace('_count', '')
                count_value = row[col]
                median_col = f'{date}_median'
                median_value = row[median_col] if median_col in df.columns else None
                reshaped_data.append([suburb, count_value, median_value, date, property_type])
    
    # Create a new DataFrame from the reshaped data
    reshaped_df = pd.DataFrame(reshaped_data, columns=['Suburb', 'Count', 'Median', 'Date', 'Property Type'])
    
    return reshaped_df

# Apply reshaping function
df_reshaped = reshape_data(df)

In [475]:
df_reshaped

,Suburb,Count,Median,Date,Property Type
0,Albert Park-Middle Park-West St Kilda,352,165,Mar 2000,1 bedroom flat
1,Albert Park-Middle Park-West St Kilda,347,165,Jun 2000,1 bedroom flat
2,Albert Park-Middle Park-West St Kilda,378,170,Sep 2000,1 bedroom flat
3,Albert Park-Middle Park-West St Kilda,369,175,Dec 2000,1 bedroom flat
4,Albert Park-Middle Park-West St Kilda,395,180,Mar 2001,1 bedroom flat
...,...,...,...,...,...
103504,Group Total,12149,360,Mar 2022,All properties
103505,Group Total,12089,365,Jun 2022,All properties
103506,Group Total,12258,380,Sep 2022,All properties
103507,Group Total,12072,380,Dec 2022,All properties


In [476]:
df_cleaned = df_reshaped[df_reshaped['Suburb'] != 'Group Total']

In [477]:
df_cleaned['Count'] = pd.to_numeric(df_cleaned['Count'], errors='coerce')
df_cleaned['Median'] = pd.to_numeric(df_cleaned['Median'], errors='coerce')

/var/folders/_j/twr8x6q52wn1cwysdlwr0lbc0000gn/T/ipykernel_7224/4123728603.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Count'] = pd.to_numeric(df_cleaned['Count'], errors='coerce')
/var/folders/_j/twr8x6q52wn1cwysdlwr0lbc0000gn/T/ipykernel_7224/4123728603.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Median'] = pd.to_numeric(df_cleaned['Median'], errors='coerce')


In [478]:
df_cleaned

,Suburb,Count,Median,Date,Property Type
0,Albert Park-Middle Park-West St Kilda,352.0,165.0,Mar 2000,1 bedroom flat
1,Albert Park-Middle Park-West St Kilda,347.0,165.0,Jun 2000,1 bedroom flat
2,Albert Park-Middle Park-West St Kilda,378.0,170.0,Sep 2000,1 bedroom flat
3,Albert Park-Middle Park-West St Kilda,369.0,175.0,Dec 2000,1 bedroom flat
4,Albert Park-Middle Park-West St Kilda,395.0,180.0,Mar 2001,1 bedroom flat
...,...,...,...,...,...
103411,Wodonga,1233.0,380.0,Mar 2022,All properties
103412,Wodonga,1267.0,390.0,Jun 2022,All properties
103413,Wodonga,1251.0,400.0,Sep 2022,All properties
103414,Wodonga,1191.0,410.0,Dec 2022,All properties


In [479]:
df_cleaned['Total_Price'] = df_cleaned['Count'] * df_cleaned['Median']
df_cleaned

/var/folders/_j/twr8x6q52wn1cwysdlwr0lbc0000gn/T/ipykernel_7224/1462032531.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Total_Price'] = df_cleaned['Count'] * df_cleaned['Median']


,Suburb,Count,Median,Date,Property Type,Total_Price
0,Albert Park-Middle Park-West St Kilda,352.0,165.0,Mar 2000,1 bedroom flat,58080.0
1,Albert Park-Middle Park-West St Kilda,347.0,165.0,Jun 2000,1 bedroom flat,57255.0
2,Albert Park-Middle Park-West St Kilda,378.0,170.0,Sep 2000,1 bedroom flat,64260.0
3,Albert Park-Middle Park-West St Kilda,369.0,175.0,Dec 2000,1 bedroom flat,64575.0
4,Albert Park-Middle Park-West St Kilda,395.0,180.0,Mar 2001,1 bedroom flat,71100.0
...,...,...,...,...,...,...
103411,Wodonga,1233.0,380.0,Mar 2022,All properties,468540.0
103412,Wodonga,1267.0,390.0,Jun 2022,All properties,494130.0
103413,Wodonga,1251.0,400.0,Sep 2022,All properties,500400.0
103414,Wodonga,1191.0,410.0,Dec 2022,All properties,488310.0


In [480]:
df_cleaned

,Suburb,Count,Median,Date,Property Type,Total_Price
0,Albert Park-Middle Park-West St Kilda,352.0,165.0,Mar 2000,1 bedroom flat,58080.0
1,Albert Park-Middle Park-West St Kilda,347.0,165.0,Jun 2000,1 bedroom flat,57255.0
2,Albert Park-Middle Park-West St Kilda,378.0,170.0,Sep 2000,1 bedroom flat,64260.0
3,Albert Park-Middle Park-West St Kilda,369.0,175.0,Dec 2000,1 bedroom flat,64575.0
4,Albert Park-Middle Park-West St Kilda,395.0,180.0,Mar 2001,1 bedroom flat,71100.0
...,...,...,...,...,...,...
103411,Wodonga,1233.0,380.0,Mar 2022,All properties,468540.0
103412,Wodonga,1267.0,390.0,Jun 2022,All properties,494130.0
103413,Wodonga,1251.0,400.0,Sep 2022,All properties,500400.0
103414,Wodonga,1191.0,410.0,Dec 2022,All properties,488310.0


In [481]:
df_cleaned[['Month', 'Year']] = df_cleaned['Date'].str.split(' ', expand=True)
df_cleaned = df_cleaned.drop('Date', axis=1)
df_cleaned['Year'] = pd.to_numeric(df_cleaned['Year'], errors='coerce')


/var/folders/_j/twr8x6q52wn1cwysdlwr0lbc0000gn/T/ipykernel_7224/1314529076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['Month', 'Year']] = df_cleaned['Date'].str.split(' ', expand=True)
/var/folders/_j/twr8x6q52wn1cwysdlwr0lbc0000gn/T/ipykernel_7224/1314529076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['Month', 'Year']] = df_cleaned['Date'].str.split(' ', expand=True)


In [482]:
df_cleaned['Year'] = df_cleaned['Year'].astype(int)

In [483]:
df_cleaned

,Suburb,Count,Median,Property Type,Total_Price,Month,Year
0,Albert Park-Middle Park-West St Kilda,352.0,165.0,1 bedroom flat,58080.0,Mar,2000
1,Albert Park-Middle Park-West St Kilda,347.0,165.0,1 bedroom flat,57255.0,Jun,2000
2,Albert Park-Middle Park-West St Kilda,378.0,170.0,1 bedroom flat,64260.0,Sep,2000
3,Albert Park-Middle Park-West St Kilda,369.0,175.0,1 bedroom flat,64575.0,Dec,2000
4,Albert Park-Middle Park-West St Kilda,395.0,180.0,1 bedroom flat,71100.0,Mar,2001
...,...,...,...,...,...,...,...
103411,Wodonga,1233.0,380.0,All properties,468540.0,Mar,2022
103412,Wodonga,1267.0,390.0,All properties,494130.0,Jun,2022
103413,Wodonga,1251.0,400.0,All properties,500400.0,Sep,2022
103414,Wodonga,1191.0,410.0,All properties,488310.0,Dec,2022


In [484]:
df_cleaned['Property Type'] = df_cleaned['Property Type'].str.split(' ').str[-1]

df_cleaned = df_cleaned.groupby(['Suburb', 'Month', 'Year', 'Property Type'], as_index=False).agg({
    'Count': 'sum', 
    'Total_Price': 'sum' 
})
df_cleaned['Median'] = df_cleaned['Total_Price'] / df_cleaned['Count']

In [485]:
df_cleaned

,Suburb,Month,Year,Property Type,Count,Total_Price,Median
0,Albert Park-Middle Park-West St Kilda,Dec,2000,flat,700.0,152126.0,217.322857
1,Albert Park-Middle Park-West St Kilda,Dec,2000,house,274.0,98998.0,361.306569
2,Albert Park-Middle Park-West St Kilda,Dec,2000,properties,1178.0,323950.0,275.000000
3,Albert Park-Middle Park-West St Kilda,Dec,2001,flat,768.0,183210.0,238.554688
4,Albert Park-Middle Park-West St Kilda,Dec,2001,house,311.0,123771.0,397.977492
...,...,...,...,...,...,...,...
40291,Yarraville-Seddon,Sep,2021,house,627.0,350847.0,559.564593
40292,Yarraville-Seddon,Sep,2021,properties,1351.0,648480.0,480.000000
40293,Yarraville-Seddon,Sep,2022,flat,606.0,225480.0,372.079208
40294,Yarraville-Seddon,Sep,2022,house,702.0,394562.0,562.054131


In [486]:
new_count = df_cleaned.groupby(['Suburb', 'Year', 'Property Type'])['Count'].sum().unstack(fill_value=0)
#new_count = df_cleaned.groupby(['Suburb', 'Month', 'Year', 'Property Type'])['Count'].sum().unstack(fill_value=0)
new_count['Count'] = new_count.get('properties', 0) - new_count.drop(columns=['properties'], errors='ignore').sum(axis=1)
new_count_result= new_count[['Count']].reset_index()

In [487]:
new_count_result

Property Type,Suburb,Year,Count
0,Albert Park-Middle Park-West St Kilda,2000,731.0
1,Albert Park-Middle Park-West St Kilda,2001,802.0
2,Albert Park-Middle Park-West St Kilda,2002,707.0
3,Albert Park-Middle Park-West St Kilda,2003,1073.0
4,Albert Park-Middle Park-West St Kilda,2004,1024.0
...,...,...,...
3499,Yarraville-Seddon,2019,566.0
3500,Yarraville-Seddon,2020,612.0
3501,Yarraville-Seddon,2021,496.0
3502,Yarraville-Seddon,2022,535.0


In [488]:
#new_price = df_cleaned.groupby(['Suburb', 'Month', 'Year', 'Property Type'])['Total_Price'].sum().unstack(fill_value=0)
new_price = df_cleaned.groupby(['Suburb', 'Year', 'Property Type'])['Total_Price'].sum().unstack(fill_value=0)

# 计算 All properties 和 Other properties 的差异
new_price['Total_Price'] = new_price.get('properties', 0) - new_price.drop(columns=['properties'], errors='ignore').sum(axis=1)

# 选择包含差异的rice结果
new_price_result= new_price[['Total_Price']].reset_index()

In [489]:
new_price_result

Property Type,Suburb,Year,Total_Price
0,Albert Park-Middle Park-West St Kilda,2000,239321.0
1,Albert Park-Middle Park-West St Kilda,2001,250986.0
2,Albert Park-Middle Park-West St Kilda,2002,222586.0
3,Albert Park-Middle Park-West St Kilda,2003,325820.0
4,Albert Park-Middle Park-West St Kilda,2004,325179.0
...,...,...,...
3499,Yarraville-Seddon,2019,353045.0
3500,Yarraville-Seddon,2020,375102.0
3501,Yarraville-Seddon,2021,296083.0
3502,Yarraville-Seddon,2022,279213.0


In [490]:
for index, row in new_price_result.iterrows():
    mask = (
        (df_cleaned['Suburb'] == row['Suburb']) &
        (df_cleaned['Year'] == row['Year']) &
        (df_cleaned['Property Type'] == 'properties')
    )
    df_cleaned.loc[mask, 'Total_Price'] = row['Total_Price']

In [491]:
for index, row in new_count_result.iterrows():
    mask = (
        (df_cleaned['Suburb'] == row['Suburb']) &
        (df_cleaned['Year'] == row['Year']) &
        (df_cleaned['Property Type'] == 'properties')
    )
    df_cleaned.loc[mask, 'Count'] = row['Count']

In [492]:
df_cleaned = df_cleaned.drop(columns=['Month'])

In [494]:
df_cleaned =  df_cleaned.drop_duplicates(subset=['Suburb', 'Year', 'Property Type'])

In [495]:
df_cleaned

,Suburb,Year,Property Type,Count,Total_Price,Median
0,Albert Park-Middle Park-West St Kilda,2000,flat,700.0,152126.0,217.322857
1,Albert Park-Middle Park-West St Kilda,2000,house,274.0,98998.0,361.306569
2,Albert Park-Middle Park-West St Kilda,2000,properties,731.0,239321.0,275.000000
3,Albert Park-Middle Park-West St Kilda,2001,flat,768.0,183210.0,238.554688
4,Albert Park-Middle Park-West St Kilda,2001,house,311.0,123771.0,397.977492
...,...,...,...,...,...,...
40093,Yarraville-Seddon,2002,house,575.0,124610.0,216.713043
40094,Yarraville-Seddon,2002,properties,296.0,76939.0,200.000000
40224,Yarraville-Seddon,2023,flat,566.0,211760.0,374.134276
40225,Yarraville-Seddon,2023,house,710.0,412660.0,581.211268


In [496]:
median_all_properties = df_cleaned.loc[df_cleaned['Property Type'] == 'properties', 'Total_Price'] / df_cleaned.loc[df_cleaned['Property Type'] == 'properties', 'Count']
df_cleaned.loc[df_cleaned['Property Type'] == 'properties', 'Median'] = median_all_properties

df_cleaned.loc[df_cleaned['Property Type'] == 'properties', 'Property Type'] = 'Other properties'

In [497]:
df_cleaned

,Suburb,Year,Property Type,Count,Total_Price,Median
0,Albert Park-Middle Park-West St Kilda,2000,flat,700.0,152126.0,217.322857
1,Albert Park-Middle Park-West St Kilda,2000,house,274.0,98998.0,361.306569
2,Albert Park-Middle Park-West St Kilda,2000,Other properties,731.0,239321.0,327.388509
3,Albert Park-Middle Park-West St Kilda,2001,flat,768.0,183210.0,238.554688
4,Albert Park-Middle Park-West St Kilda,2001,house,311.0,123771.0,397.977492
...,...,...,...,...,...,...
40093,Yarraville-Seddon,2002,house,575.0,124610.0,216.713043
40094,Yarraville-Seddon,2002,Other properties,296.0,76939.0,259.929054
40224,Yarraville-Seddon,2023,flat,566.0,211760.0,374.134276
40225,Yarraville-Seddon,2023,house,710.0,412660.0,581.211268


In [498]:
output_file_path = 'pased_data_clean.csv'
df_cleaned.to_csv(output_file_path, index=False)